In [10]:
import math
import numpy as np
import os
import pandas as pd
import pickle
import pystan

In [11]:
data_dir = '/home/jpfeil/treehouse/maris/pdx/data'

In [12]:
pth = os.path.join(data_dir, '2018-12-28-pdx-clinical-final-for-paper.txt')

pdx_clinical = pd.read_csv(pth, sep='\t')

# Only consider models that are part of the PPTC
pdx_clinical = pdx_clinical[pdx_clinical['RNA Part of PPTC'] == 'yes']

In [21]:
# Create Attributes File for TumorMap Analysis

pth = os.path.join(data_dir, 'tumormap-attr-2019-01-20-pdx.tsv')

attr = pd.DataFrame(columns=['Model', 'Histology'])

for (model, hist), rows in pdx_clinical.groupby(['Model', 'Histology-Detailed']):
    attr.loc[len(attr), :] = [model, hist]
    
attr.to_csv(pth, sep='\t', index=False)

In [14]:
# Load PDX expression data normalized to FPKM
pth = os.path.join(data_dir, 'rna-seq-2019-01-20/pdx/pdx-FPKM-2019-01-20.tsv')
fpkm = pd.read_csv(pth, sep='\t', index_col=0)
fpkm.drop('gene_short_name', axis=1, inplace=True)

In [15]:
# Convert to TPM
tpm = pd.DataFrame(index=fpkm.index, columns=fpkm.columns)
for col in fpkm.columns:
    tpm[col] = np.exp(np.log(fpkm[col]) - np.log(fpkm[col].sum()) + np.log(1E6))

/home/jpfeil/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [16]:
# 
exp = tpm

In [17]:
# Check how many of the PDXs have expression data

pdxs = pdx_clinical['Model'].values
print('Started with ', len(pdxs))

pdxs = [x for x in pdxs if x in exp.columns]
print('Ended with ', len(pdxs))

('Started with ', 248)
('Ended with ', 248)


In [18]:
# One of the MLL PDXs is missing expression data
for m in pdx_clinical['Model'].values:
    if m not in exp.columns:
        print(m)

In [19]:
# Remove genes that have more than 80% of the genes non-expressed
# Remove genes that are in the bottom 20% for low variance

def expression_variance_filter(norm_df):
    """
    This function was taken from the UCSC Treehouse protocol pipeline:
    https://github.com/UCSC-Treehouse/protocol/blob/master/3_generate-thresholds.ipynb
    """
    proportion_unexpressed = 0.8
    variance_filter_level = 0.2

    max_ok_zeroes = len(norm_df.columns) * proportion_unexpressed
    def sufficiently_expressed(series):
        return len(series[series <= 0.]) < max_ok_zeroes

    withZeroes = norm_df.apply(sufficiently_expressed, axis=1) # Series: Gene to True (keep) or False
    expression_filtered_df = norm_df[withZeroes]

    variance = expression_filtered_df.apply(np.std, axis=1) #  Series
    cut_proportion = int(math.ceil(len(variance) * variance_filter_level))
    keep_proportion = len(variance) - cut_proportion

    # list of genes that remain after
    expr_var_filtered_genelist = variance.nlargest(keep_proportion) #  Series

    return expr_var_filtered_genelist.index

In [20]:
# Subset expression matrix to the PPTC PDXs
exp = exp[pdxs]

# Apply log2(TPM + 1) transformation
exp = np.log2(exp + 1.0)

# Filter genes that are mostly zero or have 
# variance below the 20th percentile
filt_genes = expression_variance_filter(exp)
exp = exp.reindex(filt_genes)

# Center data
exp = exp.apply(lambda x: x - x.mean(), axis=1) 

# Drop missing values
exp.dropna(inplace=True)

exp.shape

pth = os.path.join(data_dir, 'tumormap-exp-2019-01-20-pdx.tsv')
exp.to_csv(pth, sep='\t')

In [11]:
pdx_clinical.groupby('Histology-Priors').count()

,DNA Part of PPTC,RNA Part of PPTC,Model Part of PPTC,Have snp file,snp array filename,snp array sample ID,Have maf,RNA human bam filename,Have fpkm file,FPKM filename,...,Histology-Oncoprints2,PI,Sex,Phase,Phase-Oncoprints,Age,Ethnicity,Site of Initial Tumor,Site of Specimen,Histopathology
Histology-Priors,,,,,,,,,,,,,,,,,,,,,
BCP-ALL,37,37,37,37,36,36,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
Brain,42,42,42,42,38,38,42,42,42,42,...,42,42,42,42,42,37,42,42,42,42
Carcinoma,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
ETP-ALL,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
Ewing Sarcoma,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
MLL-ALL,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Neuroblastoma,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,28,33,33,33,33
Osteosarcoma,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,27,32,32,32,32
Other Sarcoma,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,1,3,3,3,3


In [12]:
for i, rows in pdx_clinical.groupby(['Histology-Detailed', 'Histology-Priors']):
    print(i, len(rows))

(('ATRT', 'Rhabdoid'), 6)
(('Alveolar Rhabdomyosarcoma', 'Rhabdomyosarcoma'), 6)
(('Astrocytoma', 'Brain'), 2)
(('BCP-ALL', 'BCP-ALL'), 37)
(('CNS germinoma', 'Brain'), 1)
(('Colon Carcinoma', 'Carcinoma'), 1)
(('DIPG', 'Brain'), 2)
(('ETMR', 'Brain'), 3)
(('ETP-ALL', 'ETP-ALL'), 6)
(('Embryonal Rhabdomyosarcoma', 'Rhabdomyosarcoma'), 6)
(('Ependymoma', 'Brain'), 6)
(('Ewing Sarcoma', 'Ewing Sarcoma'), 9)
(('Glioblastoma', 'Brain'), 4)
(('High-grade glioma', 'Brain'), 1)
(('MLL-ALL', 'MLL-ALL'), 10)
(('Medulloblastoma', 'Brain'), 18)
(('Neuroblastoma', 'Neuroblastoma'), 33)
(('Osteosarcoma', 'Osteosarcoma'), 32)
(('Other Renal', 'Renal'), 2)
(('Other Sarcoma', 'Other Sarcoma'), 3)
(('PNET', 'Brain'), 5)
(('Ph+-ALL', 'Ph+/like-ALL'), 3)
(('Ph-likeALL', 'Ph+/like-ALL'), 15)
(('Rhabdoid', 'Rhabdoid'), 4)
(('Small Cell Carcinoma', 'Carcinoma'), 2)
(('T-ALL', 'T-ALL'), 19)
(('Wilms', 'Renal'), 12)


In [13]:
# Match meta data and expression to each PDX. 
tmps = []
for p in exp.columns:
    if p in pdxs:
        tmp = pd.DataFrame(columns=['sample', 
                                    'tissue', 
                                    'disease', 
                                    'gene', 
                                    'gene_id', 
                                    'expression'],
                           index=range(len(exp.index)))
        
        disease = pdx_clinical.loc[pdx_clinical['Model'] == p, 'Histology-Detailed'].item()
        
        tissue = pdx_clinical.loc[pdx_clinical['Model'] == p, 'Histology-Priors'].item()
        
        tmp['sample'] = p
        tmp['tissue'] = tissue
        tmp['disease'] = disease
        tmp['gene'] = exp.index.values
        tmp['gene_id'] = range(1, len(exp.index.values) + 1)
        tmp['expression'] = exp.loc[exp.index, p].values
        
    else:
        raise ValuError()
        
    tmps.append(tmp)

In [14]:
# Merge all of these features into one large dataframe
data = pd.concat(tmps,
                 axis=0,
                 verify_integrity=True,
                 ignore_index=True)

In [15]:
# Check that the mean is centered
for i, row in data.groupby('gene'):
    assert np.isclose(np.mean(row['expression'].values), [0])

In [16]:
# Stan only takes integer values, so match disease and tissue names 
# to an integer identifier.
disease_map = dict((d, i) for d, i in zip(data['disease'].unique(), range(1, len(data['disease'].unique()) + 1)))
disease_map

tissue_map = dict((d, i) for d, i in zip(data['tissue'].unique(), range(1, len(data['tissue'].unique()) + 1)))

In [17]:
# Add disease and tissue identifiers to dataframe
data['disease_id'] = data['disease'].map(disease_map)
data['tissue_id'] = data['tissue'].map(tissue_map)

In [18]:
# Instead of using the tissue_id, stan uses 
# the disease to tissue map.
disease_tissue_map = []
for disease in sorted(data['disease_id'].unique()):
    tissue = data.loc[data['disease_id'] == disease, 'tissue_id'].unique()[0]
    disease_tissue_map.append(tissue)

In [19]:
stan_d = {'N': data.shape[0],
          'G': len(data['gene_id'].unique()),
          'T': len(data['tissue_id'].unique()),
          'D': len(data['disease_id'].unique()),
          'genes': data['gene_id'].values,
          'tissues': disease_tissue_map,
          'diseases': data['disease_id'].values,
          'y': data['expression'].values}

In [20]:
# Write the Stan data dictionary to disk
pth = os.path.join(data_dir, 'stan-data-v8-2019-01-20.dump')
pystan.stan_rdump(stan_d, pth)

In [21]:
# Save the mapping to the original disease, tissue, and gene names
pth = os.path.join(data_dir, 'stan-data-v8-map-2019-01-20.pkl')
inv_disease_map = {v: k for k, v in disease_map.items()}
inv_tissue_map = {v: k for k, v in tissue_map.items()}

gene_mapper = {}
for gene, _id in zip(data['gene'].unique(), data['gene_id'].unique()):
    gene_mapper[_id] = gene

maps = {'tissue': inv_tissue_map,
        'disease': inv_disease_map, 
        'gene': gene_mapper}

with open(pth, 'wb') as f:
    pickle.dump(maps, f)

In [22]:
inv_disease_map

{1: 'BCP-ALL',
 2: 'MLL-ALL',
 3: 'Ph+-ALL',
 4: 'T-ALL',
 5: 'Ph-likeALL',
 6: 'Other Sarcoma',
 7: 'ATRT',
 8: 'Ewing Sarcoma',
 9: 'Neuroblastoma',
 10: 'ETP-ALL',
 11: 'Ependymoma',
 12: 'Glioblastoma',
 13: 'Medulloblastoma',
 14: 'DIPG',
 15: 'Astrocytoma',
 16: 'CNS germinoma',
 17: 'High-grade glioma',
 18: 'PNET',
 19: 'ETMR',
 20: 'Embryonal Rhabdomyosarcoma',
 21: 'Wilms',
 22: 'Alveolar Rhabdomyosarcoma',
 23: 'Small Cell Carcinoma',
 24: 'Colon Carcinoma',
 25: 'Other Renal',
 26: 'Osteosarcoma',
 27: 'Rhabdoid'}

In [23]:
# Check that the gene and gene_id values are the same for 
# all of the samples
last = None
for i, row in data.groupby('sample'):
    if last is None:
        last = row
        continue
        
    assert (row['gene'].values == last['gene'].values).all()
    assert (row['gene_id'].values == last['gene_id'].values).all()